In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

/home/ilesh-dhall/AgriVerse-Capital-One-Launchpad-2025-Hackathon/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = PyPDFLoader("data/ICAR Annual Report 2023-24-english.pdf")
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_37753/3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()

/home/ilesh-dhall/AgriVerse-Capital-One-Launchpad-2025-Hackathon/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [8]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [9]:
query = "my rice crop isn't growing well due to bad climate what to do?"
print("Answer:", rag_chain.run(query))

/tmp/ipykernel_37753/223886074.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("Answer:", rag_chain.run(query))
/home/ilesh-dhall/AgriVerse-Capital-One-Launchpad-2025-Hackathon/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer: Based on the provided text, several climate-smart interventions for rice have been identified for the village Momanpur.  However, the specific interventions are not detailed.  I don't know what to recommend for your specific situation.


In [11]:
def get_weather(query: str):
    # Placeholder function
    return "Today's rainfall is 10mm, temperature is 28°C."

In [12]:
from langchain.agents import initialize_agent, Tool

tools = [
    Tool(name="Knowledge Base", func=rag_chain.run, description="Look up agricultural advice"),
    Tool(name="Weather API", func=get_weather, description="Fetch latest weather data"),
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("Given today's rainfall, should I irrigate my wheat field?")


/tmp/ipykernel_37753/3145563484.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)




> Entering new AgentExecutor chain...
Thought: I need to know today's rainfall to determine if irrigation is necessary.  I'll check the weather API for that information.

Action: Weather API
Action Input: "rainfall today"
Observation: Today's rainfall is 10mm, temperature is 28°C.
Thought:Thought: I have the rainfall data. Now I need to determine if 10mm of rainfall is sufficient for my wheat field. I'll consult the knowledge base for wheat irrigation guidelines.

Action: Knowledge Base
Action Input: "wheat irrigation guidelines, rainfall requirements"

/home/ilesh-dhall/AgriVerse-Capital-One-Launchpad-2025-Hackathon/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Observation: The provided text mentions irrigation guidelines for wheat in a few regions.  For some varieties, it specifies "restricted irrigation" and "timely sown condition" as suitable conditions.  There's also mention of yields under rainfed (16-18 q/ha) and protective irrigation (22-24 q/ha) conditions in Karnataka's Eastern and Southern Dry Zones.  However,  specific rainfall requirements aren't detailed.
Thought:Thought:The knowledge base doesn't give a precise rainfall amount needed to avoid irrigation.  The observation suggests that irrigation can increase yield, but doesn't state a threshold for when it's necessary. I need more information to give a definitive answer.  I'll try refining my knowledge base query.

Action: Knowledge Base
Action Input: "wheat irrigation, 10mm rainfall sufficient?"

/home/ilesh-dhall/AgriVerse-Capital-One-Launchpad-2025-Hackathon/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Observation: I'm sorry, but I cannot answer this question.  The provided text gives information on Diggi-based irrigation systems and wheat yields in certain regions, but it does not contain information about whether 10mm of rainfall is sufficient for wheat irrigation.
Thought:Thought:The knowledge base is insufficient to answer the question.  I can only provide a qualified answer based on the available information.

Final Answer: Based on today's rainfall of 10mm and the lack of specific irrigation guidelines for wheat in the knowledge base, I cannot definitively say whether you should irrigate your wheat field.  10mm might be sufficient depending on factors like soil type, wheat variety, and existing soil moisture.  To make an informed decision, consult local agricultural experts or more detailed resources on wheat irrigation specific to your region and conditions.

> Finished chain.


"Based on today's rainfall of 10mm and the lack of specific irrigation guidelines for wheat in the knowledge base, I cannot definitively say whether you should irrigate your wheat field.  10mm might be sufficient depending on factors like soil type, wheat variety, and existing soil moisture.  To make an informed decision, consult local agricultural experts or more detailed resources on wheat irrigation specific to your region and conditions."